In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
os.getcwd()

'/app/experiments'

In [4]:
# folder = 'D:/MITB Analytics/2025 Aug Term/MLE/cs611-mle-mitb/assignment/data/'
folder = '/app/data/'

In [5]:
feature_clickstream = pd.read_csv(folder + "feature_clickstream.csv")
features_attributes = pd.read_csv(folder + "features_attributes.csv")
features_financials = pd.read_csv(folder + "features_financials.csv")
lms_loan_daily = pd.read_csv(folder + "lms_loan_daily.csv")

In [6]:
features_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12500 entries, 0 to 12499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer_ID    12500 non-null  object
 1   Name           12500 non-null  object
 2   Age            12500 non-null  object
 3   SSN            12500 non-null  object
 4   Occupation     12500 non-null  object
 5   snapshot_date  12500 non-null  object
dtypes: object(6)
memory usage: 586.1+ KB


In [7]:
features_attributes.dtypes

Customer_ID      object
Name             object
Age              object
SSN              object
Occupation       object
snapshot_date    object
dtype: object

clickstream_sp = feature_clickstream.head(20)
attributes_sp = features_attributes.head(20)
financials_sp = features_financials.head(20)
loans_sp = lms_loan_daily.head(20)

sample_path = "D:/MITB Analytics/2025 Aug Term/MLE/Assignment 1/samples/"
clickstream_sp.to_csv(sample_path + 'clickstream_sp.csv')
attributes_sp.to_csv(sample_path + 'attributes_sp.csv')
financials_sp.to_csv(sample_path + 'financials_sp.csv')
loans_sp.to_csv(sample_path + 'loans_sp.csv')

# Exploration

## Functions

### Load Datasets

In [8]:
def load_and_explore_datasets():
    """
    Load all datasets and perform comprehensive data exploration
    """

    datasets = {}
    filenames = ['feature_clickstream.csv', 'features_attributes.csv', 'features_financials.csv', 'lms_loan_daily.csv']
    
    for filename in filenames:
        try:
            df = pd.read_csv(folder + filename)
            datasets[filename.replace('.csv', '')] = df
            print(f"✓ {filename}: {df.shape[0]} rows, {df.shape[1]} columns")
        except Exception as e:
            print(f"✗ Error loading {filename}: {e}")
    
    return datasets

### Data Quality

In [25]:
def analyze_data_quality(datasets):
    """
    Comprehensive data quality analysis
    """  
    
    for name, df in datasets.items():
        print("\n" + "=" * 60)
        print(f"--- {name.upper()} DATASET ---")
        print("=" * 60)
        
        # Basic info
        print(f"Shape: {df.shape}")
        print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

        # Columns details
        print(f"Columns: {df.dtypes}")
        
        # Missing values
        missing_values = df.isnull().sum()
        missing_pct = (missing_values / len(df)) * 100
        missing_df = pd.DataFrame({
            'Missing_Count': missing_values,
            'Missing_Percentage': missing_pct
        })
        missing_df = missing_df[missing_df.Missing_Count > 0].sort_values('Missing_Count', ascending=False)
        
        if not missing_df.empty:
            print("\nMissing Values:")
            print(missing_df)
        else:
            print("\n✓ No missing values found")
                
        # Duplicates
        duplicates = df.duplicated().sum()
        print(f"\nDuplicate rows: {duplicates}")
        
        # Unique customers
        if 'Customer_ID' in df.columns:
            unique_customers = df['Customer_ID'].nunique()
            total_records = len(df)
            print(f"Unique customers: {unique_customers}")
            print(f"Records per customer (avg): {total_records/unique_customers:.2f}")

### Customer Coverage

In [10]:
def analyze_customer_coverage(datasets):
    """
    Analyze customer coverage across datasets
    """
    customer_sets = {}
    for name, df in datasets.items():
        if 'Customer_ID' in df.columns:
            customer_sets[name] = set(df['Customer_ID'].unique())
            print(f"{name}: {len(customer_sets[name])} unique customers")
    
    # Find intersection of all datasets
    all_datasets = list(customer_sets.keys())
    if len(all_datasets) > 1:
        intersection = customer_sets[all_datasets[0]]
        for dataset in all_datasets[1:]:
            intersection = intersection.intersection(customer_sets[dataset])
        
        print(f"\nCustomers present in ALL datasets: {len(intersection)}")
        
        # Create coverage matrix
        all_customers = set()
        for customers in customer_sets.values():
            all_customers.update(customers)
        
        coverage_matrix = []
        for customer in sorted(all_customers):
            row = [customer]
            for dataset in all_datasets:
                row.append('✓' if customer in customer_sets[dataset] else '✗')
            coverage_matrix.append(row)
        
        coverage_df = pd.DataFrame(coverage_matrix, 
                                 columns=['Customer_ID'] + all_datasets)
        print(f"\nTotal unique customers across all datasets: {len(all_customers)}")

### Temporal patterns

In [30]:
def analyze_temporal_patterns(datasets):
    """
    Analyze temporal patterns in the data
    """    
    for name, df in datasets.items():
        if 'snapshot_date' in df.columns:
            
            print("\n" + "=" * 60)
            print(f"--- {name.upper()} TEMPORAL PATTERNS ---")
            print("=" * 60)
            
            # Convert to datetime
            df['snapshot_date'] = pd.to_datetime(df['snapshot_date'])
            
            # Date range
            date_range = df['snapshot_date'].agg(['min', 'max'])
            print(f"Date range: {date_range['min'].strftime('%Y-%m-%d')} to {date_range['max'].strftime('%Y-%m-%d')}")
            
            # Unique dates
            unique_dates = df['snapshot_date'].nunique()
            print(f"Unique snapshot dates: {unique_dates}")
            
            # Records per date
            records_per_date = df['snapshot_date'].value_counts().sort_index()
            print(f"Records per date (sample):")
            print(records_per_date.head())

### Loan Default Indicators

In [26]:
def analyze_loan_default_indicators(datasets):
    """
    Identify potential loan default indicators
    """
    print("\n" + "=" * 60)
    print("LOAN DEFAULT INDICATORS ANALYSIS")
    print("=" * 60)
    
    # Analyze loans dataset for default patterns
    if 'lms_loan_daily' in datasets:
        loans_df = datasets['lms_loan_daily']
        print("\n--- LOAN PAYMENT PATTERNS ---")
        
        # Overdue analysis
        loans_df['has_overdue'] = loans_df['overdue_amt'] > 0
        overdue_summary = loans_df['has_overdue'].value_counts()
        print(f"Records with overdue amounts: {overdue_summary}")
        
        if overdue_summary.get(True, 0) > 0:
            print(f"\nOverdue amount statistics:")
            overdue_stats = loans_df[loans_df['has_overdue']]['overdue_amt'].describe()
            print(overdue_stats)
        
        # Payment ratio analysis
        loans_df['payment_ratio'] = loans_df['paid_amt'] / loans_df['due_amt'].replace(0, np.nan)
        payment_stats = loans_df['payment_ratio'].describe()
        print(f"\nPayment ratio statistics (paid/due):")
        print(payment_stats)
        
        # Identify potential defaulters
        loans_df['potential_default'] = (
            (loans_df['overdue_amt'] > 0) | 
            (loans_df['payment_ratio'] < 0.5)
        )
        default_summary = loans_df['potential_default'].value_counts()
        print(f"\nPotential default indicators:")
        print(default_summary)
    
    # Analyze financial indicators
    if 'financials' in datasets:
        fin_df = datasets['financials']
        print(f"\n--- FINANCIAL RISK INDICATORS ---")
        
        # Credit mix analysis
        if 'Credit_Mix' in fin_df.columns:
            credit_mix = fin_df['Credit_Mix'].value_counts()
            print(f"\nCredit Mix distribution:")
            print(credit_mix)
        
        # Payment behavior analysis
        if 'Payment_Behaviour' in fin_df.columns:
            payment_behavior = fin_df['Payment_Behaviour'].value_counts()
            print(f"\nPayment Behavior distribution:")
            print(payment_behavior)
        
        # Delay analysis
        if 'Delay_from_due_date' in fin_df.columns:
            delay_stats = fin_df['Delay_from_due_date'].describe()
            print(f"\nDelay from due date statistics:")
            print(delay_stats)
        
        # High-risk indicators
        risk_indicators = []
        if 'Credit_Mix' in fin_df.columns:
            risk_indicators.append(fin_df['Credit_Mix'] == 'Bad')
        if 'Delay_from_due_date' in fin_df.columns:
            risk_indicators.append(fin_df['Delay_from_due_date'] > 30)
        if 'Credit_Utilization_Ratio' in fin_df.columns:
            risk_indicators.append(fin_df['Credit_Utilization_Ratio'] > 80)
        
        if risk_indicators:
            high_risk_mask = pd.concat(risk_indicators, axis=1).any(axis=1)
            print(f"\nCustomers with high-risk indicators: {high_risk_mask.sum()}/{len(fin_df)}")


### Feature Distribution

In [38]:
def analyze_feature_distributions(datasets):
    """
    Analyze key feature distributions
    """
    
    for name, df in datasets.items():
        print("\n" + "=" * 60)
        print(f"--- {name.upper()} KEY FEATURES ---")
        print("=" * 60)
        
        # Numerical features
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            print(f"\nNumerical features summary:")
            print(df[numeric_cols].describe())
        
        # Categorical features
        categorical_cols = df.select_dtypes(include=['object']).columns
        categorical_cols = [col for col in categorical_cols 
                    if col not in ['Customer_ID', 'Name', 'SSN'] 
                    and not col.endswith('_id')]
        
        if len(categorical_cols) > 0:
            print(f"\nCategorical features:")
            for col in categorical_cols[:5]:  # Show first 5 categorical columns
                print(f"\n{col}:")
                print(df[col].value_counts().head())

### Data Issues

feature_clickstream = pd.read_csv(folder + "feature_clickstream.csv")
features_attributes = pd.read_csv(folder + "features_attributes.csv")
features_financials = pd.read_csv(folder + "features_financials.csv")
lms_loan_daily = pd.read_csv(folder + "lms_loan_daily.csv")

In [14]:
lms_loan_daily.head()

,loan_id,Customer_ID,loan_start_date,tenure,installment_num,loan_amt,due_amt,paid_amt,overdue_amt,balance,snapshot_date
0,CUS_0x1000_2023_05_01,CUS_0x1000,2023-05-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-05-01
1,CUS_0x1000_2023_05_01,CUS_0x1000,2023-05-01,10,1,10000,1000.0,1000.0,0.0,9000.0,2023-06-01
2,CUS_0x1000_2023_05_01,CUS_0x1000,2023-05-01,10,2,10000,1000.0,1000.0,0.0,8000.0,2023-07-01
3,CUS_0x1000_2023_05_01,CUS_0x1000,2023-05-01,10,3,10000,1000.0,0.0,1000.0,8000.0,2023-08-01
4,CUS_0x1000_2023_05_01,CUS_0x1000,2023-05-01,10,4,10000,1000.0,2000.0,0.0,6000.0,2023-09-01


In [15]:
def identify_data_issues(datasets):
    """
    Identify potential data quality issues
    """
    
    issues_found = []
    
    for name, df in datasets.items():
        print("\n" + "=" * 60)
        print(f"\n--- {name.upper()} ISSUES ---")
        print("=" * 60)
        
        # Check for negative values where they shouldn't be
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if 'amount' in col.lower() or 'balance' in col.lower() or 'income' in col.lower():
                negative_count = (df[col] < 0).sum()
                if negative_count > 0:
                    print(f"⚠ {col}: {negative_count} negative values")
                    issues_found.append(f"{name}.{col}: negative values")
        
        # Check for outliers using IQR method
        for col in numeric_cols:
            if col not in ['Customer_ID', 'Age']:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
                if outliers > len(df) * 0.1:  # More than 10% outliers
                    print(f"⚠ {col}: {outliers} potential outliers ({outliers/len(df)*100:.1f}%)")
                    issues_found.append(f"{name}.{col}: high outlier count")
        
        # Check for inconsistent data formats
        if 'snapshot_date' in df.columns:
            try:
                pd.to_datetime(df['snapshot_date'])
            except:
                print(f"⚠ snapshot_date: inconsistent date formats")
                issues_found.append(f"{name}.snapshot_date: format issues")
    
    if not issues_found:
        print("\n✓ No major data quality issues detected!")
    else:
        print(f"\n📋 Summary: {len(issues_found)} potential issues identified")

## Results

### Load Datasets

In [16]:
datasets = load_and_explore_datasets()

✓ feature_clickstream.csv: 215376 rows, 22 columns
✓ features_attributes.csv: 12500 rows, 6 columns
✓ features_financials.csv: 12500 rows, 22 columns
✓ lms_loan_daily.csv: 137500 rows, 11 columns


### Data Quality

In [27]:
analyze_data_quality(datasets)


--- FEATURE_CLICKSTREAM DATASET ---
Shape: (215376, 22)
Memory usage: 47731.77 KB
Columns: fe_1                      int64
fe_2                      int64
fe_3                      int64
fe_4                      int64
fe_5                      int64
fe_6                      int64
fe_7                      int64
fe_8                      int64
fe_9                      int64
fe_10                     int64
fe_11                     int64
fe_12                     int64
fe_13                     int64
fe_14                     int64
fe_15                     int64
fe_16                     int64
fe_17                     int64
fe_18                     int64
fe_19                     int64
fe_20                     int64
Customer_ID              object
snapshot_date    datetime64[ns]
dtype: object

✓ No missing values found

Duplicate rows: 0
Unique customers: 8974
Records per customer (avg): 24.00

--- FEATURES_ATTRIBUTES DATASET ---
Shape: (12500, 6)
Memory usage: 3590.60 KB
Columns

### Customer Coverage

In [18]:
analyze_customer_coverage(datasets)

feature_clickstream: 8974 unique customers
features_attributes: 12500 unique customers
features_financials: 12500 unique customers
lms_loan_daily: 12500 unique customers

Customers present in ALL datasets: 8974

Total unique customers across all datasets: 12500


### Temporal Patterns

In [31]:
analyze_temporal_patterns(datasets)


--- FEATURE_CLICKSTREAM TEMPORAL PATTERNS ---
Date range: 2023-01-01 to 2024-12-01
Unique snapshot dates: 24
Records per date (sample):
snapshot_date
2023-01-01    8974
2023-02-01    8974
2023-03-01    8974
2023-04-01    8974
2023-05-01    8974
Name: count, dtype: int64

--- FEATURES_ATTRIBUTES TEMPORAL PATTERNS ---
Date range: 2023-01-01 to 2025-01-01
Unique snapshot dates: 25
Records per date (sample):
snapshot_date
2023-01-01    530
2023-02-01    501
2023-03-01    506
2023-04-01    510
2023-05-01    521
Name: count, dtype: int64

--- FEATURES_FINANCIALS TEMPORAL PATTERNS ---
Date range: 2023-01-01 to 2025-01-01
Unique snapshot dates: 25
Records per date (sample):
snapshot_date
2023-01-01    530
2023-02-01    501
2023-03-01    506
2023-04-01    510
2023-05-01    521
Name: count, dtype: int64

--- LMS_LOAN_DAILY TEMPORAL PATTERNS ---
Date range: 2023-01-01 to 2025-11-01
Unique snapshot dates: 35
Records per date (sample):
snapshot_date
2023-01-01     530
2023-02-01    1031
2023-03-01

### Loan Default Indicators

In [20]:
analyze_loan_default_indicators(datasets)


LOAN DEFAULT INDICATORS ANALYSIS

--- LOAN PAYMENT PATTERNS ---
Records with overdue amounts: has_overdue
False    108624
True      28876
Name: count, dtype: int64

Overdue amount statistics:
count    28876.000000
mean      4151.786951
std       2341.063641
min       1000.000000
25%       2000.000000
50%       4000.000000
75%       6000.000000
max      10000.000000
Name: overdue_amt, dtype: float64

Payment ratio statistics (paid/due):
count    125000.000000
mean          0.782992
std           0.451430
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           4.000000
Name: payment_ratio, dtype: float64

Potential default indicators:
potential_default
False    108624
True      28876
Name: count, dtype: int64


### Feature Distribution

In [39]:
analyze_feature_distributions(datasets)


--- FEATURE_CLICKSTREAM KEY FEATURES ---

Numerical features summary:
                fe_1           fe_2           fe_3           fe_4  \
count  215376.000000  215376.000000  215376.000000  215376.000000   
mean      101.414796     103.096195     104.333709     105.648503   
std        99.833594      99.930002     100.599865     100.326065   
min      -378.000000    -356.000000    -399.000000    -307.000000   
25%        34.000000      36.000000      36.000000      38.000000   
50%       102.000000     103.000000     104.000000     106.000000   
75%       169.000000     171.000000     172.000000     173.000000   
max       541.000000     560.000000     583.000000     562.000000   

                fe_5           fe_6           fe_7           fe_8  \
count  215376.000000  215376.000000  215376.000000  215376.000000   
mean      106.996676     103.235922     107.070337     110.718724   
std       100.693607     100.270388     100.323265     100.243698   
min      -343.000000    -321.00

In [34]:
features_financials.head()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,snapshot_date
0,CUS_0x1000,30625.94,2706.161667,6,5,27,2,"Credit-Builder Loan, and Home Equity Loan",57,26,1.63,11.0,Bad,1562.91,30.077191,10 Years and 9 Months,Yes,42.941090,77.31427572208112,High_spent_Medium_value_payments,400.36080052211616,2023-05-01
1,CUS_0x1009,52312.68_,4250.390000,6,5,17,4,"Not Specified, Home Equity Loan, Credit-Builde...",5,18,9.73,4.0,_,202.68,40.286997,31 Years and 0 Months,Yes,108.366467,58.66019164829086,High_spent_Medium_value_payments,508.01234122645366,2025-01-01
2,CUS_0x100b,113781.38999999998,9549.782500,1,4,1,0,NaN,14,8,8.34,4.0,Good,1030.2,28.592943,15 Years and 10 Months,No,0.000000,617.0792665202719,High_spent_Small_value_payments,597.8989834797281,2024-03-01
3,CUS_0x1011,58918.47,5208.872500,3,3,17,3,"Student Loan, Credit-Builder Loan, and Debt Co...",27,13,14.42,7.0,Standard,473.14,27.829959,15 Years and 10 Months,Yes,123.434939,383.35084463651407,Low_spent_Medium_value_payments,294.1014665671429,2023-11-01
4,CUS_0x1013,98620.98,7962.415000,3,3,6,3,"Student Loan, Debt Consolidation Loan, and Per...",12,9,1.33,3.0,Good,1233.51,26.524864,17 Years and 10 Months,No,228.018084,332.3337079767732,High_spent_Medium_value_payments,485.8897083704929,2023-12-01


### Data Issues

In [22]:
identify_data_issues(datasets)



--- FEATURE_CLICKSTREAM ISSUES ---


--- FEATURES_ATTRIBUTES ISSUES ---


--- FEATURES_FINANCIALS ISSUES ---


--- LMS_LOAN_DAILY ISSUES ---
⚠ overdue_amt: 28876 potential outliers (21.0%)
⚠ payment_ratio: 30304 potential outliers (22.0%)

📋 Summary: 2 potential issues identified


## RECOMMENDATIONS

1. **CREATE TARGET VARIABLE**: Develop a binary default indicator based on overdue amounts and payment patterns,
2. **FEATURE ENGINEERING**: Create derived features like payment_ratio, days_overdue, credit_utilization_risk,
3. **TEMPORAL FEATURES**: Extract time-based features from snapshot_dates and payment histories,
4. **CUSTOMER JOURNEY**: Analyze customer behavior progression over time using clickstream data,
5. **RISK SCORING**: Combine multiple risk indicators into composite risk scores,
6. **DATA VALIDATION**: Implement data quality checks for negative amounts and outliers,
7. **FEATURE SELECTION**: Use correlation analysis and feature importance for model preparation

# Cleaning